# Peta Timeframe untuk Strategi EMA112, VWAP, dan ML

Notebook ini merangkum bagaimana strategi EMA112 (trend-following), VWAP (mean-reversion),
dan varian sinyal ML dievaluasi di beberapa timeframe. Gunakan tabel di bawah sebagai titik
awal memilih notebook yang sesuai serta mengingatkan lokasi data/output masing-masing.

## Ringkasan Notebook Per Timeframe

| Notebook | Timeframe | Dataset | Folder Output | Catatan |
| --- | --- | --- | --- | --- |
| `strategy_comparison_tf1h.ipynb` | 1H | `data/BINANCE_ETHUSDT.P, 60.csv` | `outputs/strategy_comparison/` | Perbandingan EMA112 vs VWAP + uji sinyal ML intraday.
| `strategy_comparison_tf1d.ipynb` | 1D | `data/BINANCE_ETHUSD, 1D.csv` | `outputs/result-test/` (Excel) & `outputs/macd_daily_*.json` | Versi daily untuk melihat konsistensi tren/mean-reversion.
| (Notebook ini) `strategy_timeframe_matrix.ipynb` | N/A | — | — | Dokumentasi singkat mengenai cakupan timeframe, strategi, dan temuan pengujian. |

Semua notebook tetap kompatibel dengan sinyal ML eksternal melalui `ML_PREDICTION_PATHS` sehingga hasil model dapat diuji berdampingan dengan strategi rule-based.

## Cara Memperluas Uji Timeframe

1. Duplikasi salah satu notebook di atas dan ganti `DATA_PATH`, `OUTPUT_DIR`, serta `output_prefix`
   sesuai pasangan simbol/timeframe baru.
2. Perbarui nilai `BARS_PER_DAY` dan `BARS_PER_YEAR` agar perhitungan CAGR/volatilitas tahunan
   tetap akurat.
3. Simpan hasil backtest ke folder unik supaya artefak antar time frame tidak bercampur.
4. Ekspor seluruh tabel ke Excel (lihat blok `export_tables_to_excel`) untuk memudahkan komparasi
   lintas timeframe di satu file gabungan.

## Hasil Uji Berbasis Artefak
Angka di bawah langsung diambil dari file metrics yang dihasilkan notebook terkait (`outputs/strategy_comparison/*metrics.json` untuk 1H, `outputs/macd_daily_metrics.json` untuk 1D, dan `outputs/result-test/ml_baseline.xlsx` untuk sinyal ML hourly).

| Strategi | Timeframe | Dataset | CAGR | Sharpe | Max Drawdown | Sumber | Catatan Kinerja |
| --- | --- | --- | --- | --- | --- | --- | --- |
| EMA112 (trend-following) | 1H | `data/BINANCE_ETHUSDT.P, 60.csv` | 45.9% | 1.12 | -27.4% | `outputs/strategy_comparison/ema112_hourly_metrics.json` | CAGR intraday positif dengan Sharpe > 1, tetapi drawdown panjang dan win rate rendah karena hanya long mengikuti tren. |
| VWAP (mean-reversion) | 1H | `data/BINANCE_ETHUSDT.P, 60.csv` | 93.0% | 1.63 | -15.8% | `outputs/strategy_comparison/vwap_hourly_metrics.json` | Sharpe tertinggi dan drawdown paling dangkal; kombinasi long/short mean-reversion menjaga profit factor > 2.4. |
| ML LightGBM (signal Sharpe) | 1H | `outputs/result-test/ml_baseline.xlsx` | — | 0.72 (signal) | — | `outputs/strategy_comparison/ml_hourly_metrics.json` | Walk-forward LightGBM memberi `test_signal_sharpe` 0.72 dengan `test_accuracy` 52.8%/AUC 0.531; horizon label 5 jam. |
| MACD (trend-following) | 1D | `data/OKX_ETHUSDT.P, 1D.csv` | -16.0% | -0.67 | -27.2% | `outputs/macd_daily_metrics.json` | Performa harian negatif pada sampel ini; CAGR turun dan Sharpe < 0, perlu penyesuaian parameter atau filter tren. |


## Panduan pemilihan strategi per kondisi tren (berdasarkan hasil di atas)
- **Intraday bullish / volatilitas dengan arah jelas (1H)**: EMA112 tetap valid karena CAGR 45.9% dan Sharpe 1.12 meski drawdown dalam. Cocok bila ingin mengikuti tren panjang dengan posisi long saja.
- **Intraday sideways atau mean-reversion (1H)**: VWAP unggul dengan Sharpe 1.63 dan drawdown -15.8%. Gunakan ketika harga sering kembali ke nilai rata-rata dan siap membuka posisi long/short.
- **Harian (1D)**: MACD dengan setelan sekarang menunjukkan CAGR -16% dan Sharpe negatif. Untuk konteks bearish panjang, perlu penyesuaian threshold/parameter atau filter tren tambahan sebelum dipakai.

Lakukan re-run notebook sumber bila ada data baru supaya tabel ini otomatis mengikuti metrik terbaru.
